In [1]:
import os
import sys
from pyspark import SparkContext, SparkConf
from pyspark.sql import SQLContext

In [2]:
conf = SparkConf().setAppName('G101HW1').setMaster("local[*]")
sc = SparkContext(conf=conf)

22/04/05 21:34:14 WARN Utils: Your hostname, eunoia resolves to a loopback address: 127.0.1.1; using 192.168.185.111 instead (on interface wlp1s0)
22/04/05 21:34:14 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/04/05 21:34:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [132]:
""" 
    K = <int> Number of Partitions
    H = <int> Number of Products with Highest Popularity
    S = <str> Name of the Country -- 'all' Means all Countries
    dataset_path = Path of the Dataset File

"""
K=4
H=5
# S='United_Kingdom'
S = 'all'
# S = 'Italy'
# dataset_path = 'sample_50.csv'
# dataset_path = 'sample_10000.csv'
dataset_path = 'full_dataset.csv'
# sqlContext = SQLContext(sc)
rawData = sc.textFile(dataset_path,K).cache()
rawData.repartition(K)

MapPartitionsRDD[318] at coalesce at NativeMethodAccessorImpl.java:0

In [133]:
rawData.count()

406829

In [134]:
rawData.getNumPartitions()

4

In [135]:
def productCustomer(row, country='all'):
    """
    row = [0:TransactionID, 1:ProductID, 2:Description, 3:Quantity, 4:InvoiceDate, 5:UnitPrice, 6:CustomerID, 7:Country]
    """
    s = row.split(',')
    if int(s[3]) > 0:
        if country == 'all':
            return ((s[1], int(s[6])),0)
        elif s[7] == country:
            return ((s[1], int(s[6])),0)

In [141]:
product_customer = (rawData.map(lambda row: productCustomer(row, S)).filter(lambda row: row)
                    .groupByKey()
                    .map(lambda x: x[0]))
print(f'Product-Customer Pairs = {product_customer.count()}')

Product-Customer Pairs = 266802


In [142]:
def productCustomer2(row, country='all'):
    """
    row = [0:TransactionID, 1:ProductID, 2:Description, 3:Quantity, 4:InvoiceDate, 5:UnitPrice, 6:CustomerID, 7:Country]
    """
    s = row.split(',')
    if int(s[3]) > 0:
        if country == 'all':
            return [((s[1], int(s[6])),0)]
        elif s[7] == country:
            return [((s[1], int(s[6])),0)]
        else:
            return []
    else:
        return []

In [143]:
product_customer2 = (rawData.flatMap(lambda row: productCustomer2(row, S))
                    .groupByKey()
                    .map(lambda x: x[0]))

In [144]:
product_customer2.count()

266802

In [144]:
product_customer2.count()

266802

In [140]:
product_popularity1 = (product_customer
                        .groupByKey()
                        .mapValues(len))
product_popularity1.count()

3665

In [127]:
product_popularity1.collect()

[('22961', 2),
 ('22960', 4),
 ('22969', 1),
 ('21733', 2),
 ('84879', 1),
 ('22632', 1)]

In [61]:
product_popularity2 = (product_customer.map(lambda x:(x[0],1))
                       .reduceByKey(lambda x,y:x+y)
)
product_popularity2.count()

603

In [62]:
product_popularity2.collect()

[('22464', 10),
 ('20974', 11),
 ('22834', 11),
 ('22837', 13),
 ('22776', 21),
 ('22086', 23),
 ('20679', 18),
 ('22890', 7),
 ('21479', 7),
 ('21485', 20),
 ('22149', 20),
 ('82582', 7),
 ('22382', 35),
 ('21756', 12),
 ('84970L', 20),
 ('22189', 19),
 ('22900', 11),
 ('22168', 4),
 ('22698', 36),
 ('21843', 17),
 ('22274', 9),
 ('22457', 50),
 ('82483', 22),
 ('22112', 34),
 ('20724', 29),
 ('22720', 62),
 ('20727', 32),
 ('21673', 16),
 ('22722', 37),
 ('21928', 22),
 ('20969', 12),
 ('22970', 16),
 ('22197', 39),
 ('84378', 15),
 ('22966', 20),
 ('22791', 23),
 ('21080', 31),
 ('21781', 6),
 ('21524', 12),
 ('20749', 14),
 ('15056BL', 18),
 ('21891', 14),
 ('21746', 10),
 ('22087', 15),
 ('22139', 25),
 ('84992', 9),
 ('48187', 25),
 ('22767', 15),
 ('22386', 30),
 ('20711', 11),
 ('21523', 24),
 ('22082', 14),
 ('21982', 9),
 ('22333', 13),
 ('22971', 9),
 ('85014B', 11),
 ('21975', 22),
 ('20975', 8),
 ('22620', 11),
 ('22626', 7),
 ('21929', 19),
 ('22273', 14),
 ('20914', 32),

In [145]:
product_customer2.count()

266802